# Pricing exotic options in Python

### - Sergio Garcia, PhD

## 1- Pricing of Average Options

###### Exotic options are complex financial instruments which have the following features:
###### - Non-standard underlying developed for particular investors (banks, companies, investment funds,...)
###### - Traded over-the-counter (OTC)
###### - Have several triggers that determine the payoffs
###### - Are not priced (necessarilly) using standard models like the Black-Scholes-Merton model.

###### Example:

##### The manager of an oil refinery in Mexico wants to hedge the potential oil price increases of crude oil. The financial department of the company recommends to buy an option to hedge price jumps. The company is a large consumer of oil and it is interested in the average oil price, rather than the terminal one. Therefore, the acquisition of the vanilla call option would not help, as its payoff dependes on the terminal price (and strike). The company decides to buy an average option with the following features:

###### - An average option is a type of Asian option (a basic form of exotic options).
###### - The payoff is determined by the average underlying price for a period of time
###### - It is cheaper than the Europan or American options.
###### - The variation of the average is much smaller than the terminal price.
###### - Average can be arithmetic or geometric
###### - These are the payoffs for an Average Asian Call and Put options:

$ Payoff(Call) = Max(P_{average} - X, 0) \\
 Payoff(Put) = Max(X - P_{average}, 0)$

###### Python program for an Asian option with arithmetic average price with the following data:

In [1]:
import numpy as np
import scipy as sp    # Import libraries to use

In [2]:
s0= 40     # Today's stock price
x= 40      # Strike/Exercise price
T= 0.5     # Maturity (in years)
r= 0.05    # Risk-free rate
sigma= 0.2 # Annualized volatility

In [3]:
n_simulation= 100 # Number of simulations and steps
n_steps= 100
dt= T/n_steps

In [4]:
call= sp.zeros([n_simulation], dtype=float)
for j in range(0,n_simulation):
    sT=s0
    total=0
    for i in range(0,int(n_steps)):
        e = sp.random.normal()
        sT*= sp.exp((r-0.5*sigma**2)*dt+sigma*e*sp.sqrt(dt))
        total+=sT
    price_average = total/n_steps
    call[j]=max(price_average-x,0)

call_price=np.mean(call)*np.exp(-r*T)
print('Call price = ', round(call_price,3))

Call price =  1.428


## 2 - Pricing and 'up-and-out' Exotic Option

###### An 'up-and-out' option is a type of barrier option. Specifically, the underlying price starts from below a barrier level. Once it reaches the barrier it is knocked out. The next example prices an 'up-and-out' barrier option with an European Call:

In [5]:
import scipy as sp
import numpy as np

In [6]:
def bs_call(S,X,T,rf,sigma):
    """
       Returns: Call value under Black-Schole-Merton option model
       Format   : bs_call(S,X,T,r,sigma)
               S: current stock price
               X: exercise price
               T: maturity date in years
              rf: risk-free rate (continusouly compounded)
           sigma: volatiity of underlying security 
    """    
    from scipy import log,exp,sqrt,stats
    d1=(log(S/X)+(rf+sigma*sigma/2.)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)
    return S*stats.norm.cdf(d1)-X*exp(-rf*T)*stats.norm.cdf(d2)

In [8]:
def up_and_out_call(s0,x,T,r,sigma,n_simulation,barrier):
    """
        Returns: Call value of an up-and-out barrier option with European call
    """
    n_steps= 100 # Define number of steps.
    dt = T/n_steps
    total=0
    for j in range(0,n_simulation):
        sT=s0
        out=False
        for i in range(0,int(n_steps)):
            e= sp.random.normal()
            sT*=sp.exp((r-0.5*sigma**2)*dt+sigma*e*sp.sqrt(dt))
            if sT>barrier:
                out=True
        
        if out==False:
            total+=bs_call(s0,x,T,r,sigma)
    return total/n_simulation

4214.529275126101

###### We simulate 100 times the stock movement. For each simulation, 100 steps. Once, at some point, the stock price reaches the barrier, the payoff of the option will be zero. Otherwise, the value will be that of an European call under BS. The final value will be the mean value of all call prices that are not knocked out. Let's try the results with the following data:

In [8]:
s0= 40              # Stock price today
x= 40               # Strike price
barrier = 42        # Barrier level
T= 0.5              # Maturity in years
r=0.05              # Risk-free rate
sigma=0.2           # Annualized volatility
n_simulation = 100  # number of simulations

In [9]:
result = up_and_out_call(s0,x,T,r,sigma,n_simulation,barrier)
print('Price for the Up-and-out Call = ', round(result,3))

Price for the Up-and-out Call =  0.909
